So Late So Soon -- Age distributions depending on event times 
=======

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import seaborn as sns
%matplotlib inline

from scipy import sparse
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,FunctionTransformer

#path to data and features
DATA_PATH = "../input/"

As a first step, we load the events data that contains the information when an event was  fired. We are only interested in the timestamp and the device-id. The timestamp column is converted to datetime.

In [ ]:
events = pd.read_csv('{0}events.csv'.format(DATA_PATH)).loc[:, ['timestamp', 'device_id']]
events['timestamp'] = pd.to_datetime(events['timestamp'])

We represent the time of day in fractional hours.

In [ ]:
def fract_hour(time):
    return time.hour + time.minute / 60.0 + time.second / 3600.0
events['hour'] = events['timestamp'].apply(lambda time: fract_hour(time))

Event distribution over time
-------



Before delving into any more refined analysis, we visualize the distribution of the time stamps. Since there is a substantial amount of activity at 12 PM, we recenter the time axis for the plot. More precisely, 10 PM is interpreted as -2 AM.

In [ ]:
events['hour_recentred'] = [((time + 2) % 24)-2 for time in events['hour']]

In [ ]:
ax1 = sns.distplot(events['hour_recentred'])

ax1.set_xlim(xmin = -2, xmax = 22)
ax1.set_xlabel('Hour of day')
ax1.set_title('Events by hour -- recentered')

We see event peeks in the morning at around 10 AM and in the evening at around 9 PM. The kernel density plot is confusing at the boundaries of the domain. 

Age distribution by time of day
-------



After having developed a rough idea on the general distribution of the event timing, we now investigate connections to the user demographics. For this, we subdivide the day into 6 periods and investigate the age distribution within each of these periods. 

In order to use the demographics data, we need to join the age_train dataset with the events dataset on the 'device-id' field.

In [ ]:
age_sex = pd.read_csv('{0}gender_age_train.csv'.format(DATA_PATH)).drop('group', axis = 1)
age_sex_event = age_sex.merge(events, 'inner', on = 'device_id').drop_duplicates().drop('device_id', axis = 1)

Now, we subdivide the day into 4 bins and generate violin plots for each time interval.

In [ ]:
age_sex_event['bin'] = pd.cut(age_sex_event['hour_recentred'], [-2, 2, 7, 22])

ax = sns.violinplot(x="bin", y="age", data = age_sex_event)
ax.set_ylim(ymin = 18, ymax = 55)
ax.set_xlabel('Time of day')
ax.set_title('Age distribution by time of day')

The violin plots are fairly similar, but we do observe that the late night sessions (22 PM to 2 AM) and early morning events (2 AM to 7 AM) correspond to younger and older age groups, respectively.The violin plots are fairly similar, but we do observe that the late night sessions (22 PM to 2 AM) and early morning events (2 AM to 7 AM) correspond to younger and older age groups, respectively.

Age and gender distribution by time of day
-------

Finally, we take into account not only the age but also the gender. We see that for late-night activities, the female age median is smaller than the male age median, whereas the situation is reversed in the early morning.

In [ ]:
ax_violin = sns.violinplot(x='bin', y='age', hue = 'gender', split = False, data = age_sex_event)

ax_violin.set_ylim(ymin = 18, ymax = 55)
ax_violin.set_xlabel('Time of day')
ax_violin.set_title('Age distribution by time of day and gender')
ax_violin.legend(bbox_to_anchor=(1.05, 1), loc=2)